Along this exercise you will focus on one of them. Let’s imagine the IMDb platform wants an automatic
procedure to assign a numerical value to a user review, based of the content of the text, to identify the rate
he or she would have given to the movie. Simplifying the task, you might want to assign a label, Positive
or Negative, to identify the overall sentiment, i.e. whether the user liked the movie or not.
To do so, you will implement the TF-IDF weighting scheme on IMDb reviews. Then, you will compute
the distance between a review whose sentiment is unknown and the two sets of positive and negative
comments, assigning the sentiment of the closest one among the two.

In [16]:
import csv
reviews, labels = [], []
header = []
with open('imdb_reviews_train.csv', encoding='utf-8') as f:
   csv_reader = csv.reader(f)
   header = next(csv_reader)
   for row in csv_reader:
        reviews.append(row[0])
        labels.append(row[1])



In [17]:
from collections import Counter

print("Number of reviews in the dataset:", len(reviews))
print("Number of 1's and 0's:", [(k, v) for k, v in Counter(labels).items()])

Number of reviews in the dataset: 25000
Number of 1's and 0's: [('1', 12500), ('0', 12500)]


Apply the tokenization function listed below to your reviews. Please refer to the function’s docstring 1
for the input and output parameters. The tokenization procedure splits each comment in tokens (i.e.
separate words)


In [18]:
import string
def tokenize(docs):
#Compute the tokens for each document.
#Input: a list of strings. Each item is a document to tokenize.
#Output: a list of lists. Each item is a list containing the tokens of the
#relative document.
    tokens = []
    for doc in docs:
        for punct in string.punctuation:
            doc = doc.replace(punct, " ")
        split_doc = [ token.lower() for token in doc.split(" ") if token ]
        tokens.append(split_doc)
    return tokens

token_list = tokenize(reviews)

token_list is now a list of lists. The ith item of the outer list is a list containing all the words found in the ith review. Notice that here duplicates can be present. For example:

In [19]:
print(token_list[0])

['for', 'a', 'movie', 'that', 'gets', 'no', 'respect', 'there', 'sure', 'are', 'a', 'lot', 'of', 'memorable', 'quotes', 'listed', 'for', 'this', 'gem', 'imagine', 'a', 'movie', 'where', 'joe', 'piscopo', 'is', 'actually', 'funny', 'maureen', 'stapleton', 'is', 'a', 'scene', 'stealer', 'the', 'moroni', 'character', 'is', 'an', 'absolute', 'scream', 'watch', 'for', 'alan', 'the', 'skipper', 'hale', 'jr', 'as', 'a', 'police', 'sgt']


At this point we need to compute the term-frequency (TF) of each token (read word or term) within its document (read review). As usual, you can define a function for that.

In [20]:
def compute_TF(token_list):
    TF = []
    for document in token_list:
        tf = {}
        for token in document:
            tf[token] = tf.get(token, 0) + 1
        TF.append(tf)
    return TF

TF_list = compute_TF(token_list)

In [21]:
print(TF_list[0])

{'for': 3, 'a': 5, 'movie': 2, 'that': 1, 'gets': 1, 'no': 1, 'respect': 1, 'there': 1, 'sure': 1, 'are': 1, 'lot': 1, 'of': 1, 'memorable': 1, 'quotes': 1, 'listed': 1, 'this': 1, 'gem': 1, 'imagine': 1, 'where': 1, 'joe': 1, 'piscopo': 1, 'is': 3, 'actually': 1, 'funny': 1, 'maureen': 1, 'stapleton': 1, 'scene': 1, 'stealer': 1, 'the': 2, 'moroni': 1, 'character': 1, 'an': 1, 'absolute': 1, 'scream': 1, 'watch': 1, 'alan': 1, 'skipper': 1, 'hale': 1, 'jr': 1, 'as': 1, 'police': 1, 'sgt': 1}


Now that we have the TF for each token in each document, we can compute the inverse-term-frequency (IDF). This number is assigned to each distinct token found in the whole collection of documents and inversely weights its presence among the documents, i.e.:

 tokens that appear in only one document will have IDFt=logN
 
 tokens that appear in every document will have IDFt=0




In [22]:
import math

def compute_IDF(TF_list):
    IDF = {}
    N = len(TF_list)

    # compute the document-frequency (DF), i.e. the number of documents in which each token appears at least once 
    for review_TF in TF_list:
        for token, token_tf in review_TF.items():
            IDF[token] = IDF.get(token,0)+1
    return {token: math.log(N / df) for token,df in IDF.items()}

IDF_dict = compute_IDF(TF_list)        


In [23]:
sorted_view = sorted(IDF_dict.items(), key=lambda item: item[1])
sorted_view[:10]

[('the', 0.008314469604085238),
 ('a', 0.03351541933781697),
 ('and', 0.03401190259170586),
 ('of', 0.05226218466281087),
 ('to', 0.06293979977387414),
 ('this', 0.09924591465797242),
 ('is', 0.1086102347240488),
 ('it', 0.11536595914077863),
 ('in', 0.12606221366364628),
 ('that', 0.20722099077039452)]

The final step requires the computation of the term-frequency inverse-term-frequency (TF-IDF). It is the effective weighting scheme that can be used to compute the similarity between two documents. As usual, let's define a function for that:

In [24]:
def compute_TFIDF(TF_list, IDF_dict):
    TFIDF = []

    for doc in TF_list:
        tfidf = {}
        for token, token_tf in doc.items():
            tfidf[token] = token_tf * IDF_dict[token]
        TFIDF.append(tfidf)
    return TFIDF
tf_idf = compute_TFIDF(TF_list, IDF_dict)

In [25]:
print(tf_idf[0])

{'for': 1.005075851886905, 'a': 0.16757709668908488, 'movie': 0.9813925049416498, 'that': 0.20722099077039452, 'gets': 2.257229391273248, 'no': 1.1141321003261466, 'respect': 3.9845936982629815, 'there': 0.837387134278689, 'sure': 2.3530366364901436, 'are': 0.5868431101899066, 'lot': 2.0319474551515233, 'of': 0.05226218466281087, 'memorable': 3.6936910111111585, 'quotes': 5.5940316106970815, 'listed': 5.339139361068292, 'this': 0.09924591465797242, 'gem': 4.291820366787733, 'imagine': 3.587045148232668, 'where': 1.655900786844441, 'joe': 4.137669686960474, 'piscopo': 7.418580902748128, 'is': 0.3258307041721464, 'actually': 1.982532640511814, 'funny': 2.0743346043116913, 'maureen': 6.437751649736401, 'stapleton': 7.561681746388801, 'scene': 1.8767946184246356, 'stealer': 7.487573774235079, 'the': 0.016628939208170476, 'moroni': 8.740336742730447, 'character': 1.641547966352334, 'an': 0.7166205367455873, 'absolute': 4.315490110873637, 'scream': 4.706096104578052, 'watch': 1.5199629060064

In [26]:

def norm(d):
    return sum([value**2 for t, value in d.items()])**.5

def dot_product(v1, v2):
    # only the words that appear in at least one of the two vectors/documents are involved
    dict_d = set(list(v1.keys()) + list(v2.keys()))
    return sum([(v1.get(d, 0.0) * v2.get(d, 0.0)) for d in dict_d])

def cosine_similarity(v1, v2):
    return dot_product(v1, v2) / (norm(v1) * norm(v2))





We can now address our simple sentiment analysis task. Given a document with unknown label (it can be either positive or negative), we are asked to infer it from the content of the text.

A simple assumption could be made: reviews with the same sentiment share the same set of words and their usage or, in other terms, they share the same language. Although it seems a weak call, we can try to assign the labels to the reviews we have and count how many of them were correct. In order to accomplish the goal, we have to:


    split the collection in two sets, each containing only the positive and the negative reviews;

    compute the similarity between the considered review (the test document) and the two sets. You will see that there exist many methods to obtain this measure. For now, we will average the similarity between the test review and all the reviews, separately for the two sets;
    
    assign to the test review the label of the most similar set (i.e. the one with the highest average similarity).


In [27]:
pos_reviews_indices = [i for i, label in enumerate(labels) if label == '1']
neg_reviews_indices = [i for i, label in enumerate(labels) if label == '0']
len(pos_reviews_indices), len(neg_reviews_indices)

(12500, 12500)

We can now compute the similarities and then assign the class label accordingly. In order to optimize the evaluation we can exploit the fact that the cosine similarity is a commutative operator, i.e. cos(V1,V2)=cos(V2,V1). In this sense, we can encode similarities between each pair of reviews in a symmetric matrix and then use it to compute the average with respect to the positive and negative sets.

In [28]:
import numpy as np

similarities = []
y_true = labels
y_pred = []
r_len = len(tf_idf)

try:
    for i,r1 in enumerate(tf_idf):
        store_sim = []
        curr_sim = []
        for j, r2 in enumerate(tf_idf):
            if j == i:
                curr_sim.append(-1)
            elif j < i:
                curr_sim.append(similarities[j][i-j-1])
            else:
                s = cosine_similarity(tf_idf[j], tf_idf[i])
                store_sim.append(s)
                curr_sim.append(s)
        similarities.append(store_sim)
        if i in pos_reviews_indices:
            p_mask = pos_reviews_indices.copy()
            p_mask.pop(i)
            n_mask = neg_reviews_indices
        else:
            p_mask = pos_reviews_indices
            n_mask = neg_reviews_indices.copy()
            n_mask.pop(i)
        p_mean = np.array(curr_sim)[p_mask].mean()
        n_mean = np.array(curr_sim)[n_mask].mean()   
        if p_mean > n_mean:
            y_pred.append('1')
        else:
            y_pred.append('0')
            
        print(f'{100*i/(r_len):.2f}%', end='\r')
except KeyboardInterrupt:
    print('\nInterrupted')
    pred_c = len(y_pred)
    correct = sum([1 for t, p in zip(y_pred, y_true[:pred_c]) if t == p])
    print(f'Computed {i} reviews up to now. Accuracy: {correct/pred_c * 100:.2f}%')


0.50%
Interrupted
Computed 127 reviews up to now. Accuracy: 72.44%
